# Hyperparameter Tuning
*(Note: This notebook runs significantly faster if you have access to a GPU. Use either the GPUHub, Google Colab, or your own GPU.)*

In this project, you will optimize the hyperparameters of a model in 3 stages.

## Paraphrase Detection
We finetune [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased) on [MRPC](https://huggingface.co/datasets/glue/viewer/mrpc/train), a paraphrase detection dataset. This notebook is adapted from a [PyTorch Lightning example](https://lightning.ai/docs/pytorch/1.9.5/notebooks/lightning_examples/text-transformers.html).

In [ ]:
%pip install -q torch transformers lightning datasets wandb evaluate ipywidgets optuna

Note: you may need to restart the kernel to use updated packages.


The next 4 cells are:
* Imports
* The `GLUEDataModule` loads the task's dataset and creates dataloaders for the train and valid sets.
* The `GLUETransformer` implements the model forward pass and the training/validation steps. You can check here what is logged with the `self.log` calls.
* The last cell runs training with the given parameters.

In [1]:
from datetime import datetime
from typing import Optional

import wandb
import datasets
import evaluate
import lightning as L
import torch
import optuna
from optuna.pruners import NopPruner
from optuna.samplers import TPESampler
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    get_constant_schedule_with_warmup,
    get_cosine_schedule_with_warmup
)

In [2]:
wandb.login()

wandb: Currently logged in as: jan-wahli (jan-wahli-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
class GLUEDataModule(L.LightningDataModule):
    task_text_field_map = {
        "cola": ["sentence"],
        "sst2": ["sentence"],
        "mrpc": ["sentence1", "sentence2"],
        "qqp": ["question1", "question2"],
        "stsb": ["sentence1", "sentence2"],
        "mnli": ["premise", "hypothesis"],
        "qnli": ["question", "sentence"],
        "rte": ["sentence1", "sentence2"],
        "wnli": ["sentence1", "sentence2"],
        "ax": ["premise", "hypothesis"],
    }

    glue_task_num_labels = {
        "cola": 2,
        "sst2": 2,
        "mrpc": 2,
        "qqp": 2,
        "stsb": 1,
        "mnli": 3,
        "qnli": 2,
        "rte": 2,
        "wnli": 2,
        "ax": 3,
    }

    loader_columns = [
        "datasets_idx",
        "input_ids",
        "token_type_ids",
        "attention_mask",
        "start_positions",
        "end_positions",
        "labels",
    ]

    def __init__(
        self,
        model_name_or_path: str,
        task_name: str = "mrpc",
        max_seq_length: int = 128,
        train_batch_size: int = 32,
        eval_batch_size: int = 32,
        **kwargs,
    ):
        super().__init__()
        self.model_name_or_path = model_name_or_path
        self.task_name = task_name
        self.max_seq_length = max_seq_length
        self.train_batch_size = train_batch_size
        self.eval_batch_size = eval_batch_size

        self.text_fields = self.task_text_field_map[task_name]
        self.num_labels = self.glue_task_num_labels[task_name]
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name_or_path, use_fast=True)

    def setup(self, stage: str):
        self.dataset = datasets.load_dataset("glue", self.task_name)

        for split in self.dataset.keys():
            self.dataset[split] = self.dataset[split].map(
                self.convert_to_features,
                batched=True,
                remove_columns=["label"],
            )
            self.columns = [c for c in self.dataset[split].column_names if c in self.loader_columns]
            self.dataset[split].set_format(type="torch", columns=self.columns)

        self.eval_splits = [x for x in self.dataset.keys() if "validation" in x]

    def prepare_data(self):
        datasets.load_dataset("glue", self.task_name)
        AutoTokenizer.from_pretrained(self.model_name_or_path, use_fast=True)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.train_batch_size, shuffle=True)

    def val_dataloader(self):
        if len(self.eval_splits) == 1:
            return DataLoader(self.dataset["validation"], batch_size=self.eval_batch_size)
        elif len(self.eval_splits) > 1:
            return [DataLoader(self.dataset[x], batch_size=self.eval_batch_size) for x in self.eval_splits]

    def test_dataloader(self):
        if len(self.eval_splits) == 1:
            return DataLoader(self.dataset["test"], batch_size=self.eval_batch_size)
        elif len(self.eval_splits) > 1:
            return [DataLoader(self.dataset[x], batch_size=self.eval_batch_size) for x in self.eval_splits]

    def convert_to_features(self, example_batch, indices=None):
        # Either encode single sentence or sentence pairs
        if len(self.text_fields) > 1:
            texts_or_text_pairs = list(zip(example_batch[self.text_fields[0]], example_batch[self.text_fields[1]]))
        else:
            texts_or_text_pairs = example_batch[self.text_fields[0]]

        # Tokenize the text/text pairs
        features = self.tokenizer.batch_encode_plus(
            texts_or_text_pairs, max_length=self.max_seq_length, padding="max_length", truncation=True
        )

        # Rename label to labels to make it easier to pass to model forward
        features["labels"] = example_batch["label"]

        return features

In [4]:
class GLUETransformer(L.LightningModule):
    def __init__(
        self,
        model_name_or_path: str,
        num_labels: int,
        task_name: str,
        learning_rate: float = 2e-5,
        lr_schedule_type: str = "linear",
        warmup_steps: int = 0,
        weight_decay: float = 0.0,
        train_batch_size: int = 32,
        eval_batch_size: int = 32,
        gradient_clip_val: float = 1.0,
        eval_splits: Optional[list] = None,
        **kwargs,
    ):
        super().__init__()

        self.save_hyperparameters()

        self.config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, config=self.config)
        self.metric = evaluate.load(
            "glue", self.hparams.task_name, experiment_id=datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
        )

        self.validation_step_outputs = []

    def forward(self, **inputs):
        return self.model(**inputs)

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs[0]
        return loss

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        outputs = self(**batch)
        val_loss, logits = outputs[:2]

        if self.hparams.num_labels > 1:
            preds = torch.argmax(logits, axis=1)
        elif self.hparams.num_labels == 1:
            preds = logits.squeeze()

        labels = batch["labels"]
        self.validation_step_outputs.append({"loss": val_loss, "preds": preds, "labels": labels})
        return val_loss

    def on_validation_epoch_end(self):
        if self.hparams.task_name == "mnli":
            for i, output in enumerate(self.validation_step_outputs):
                # matched or mismatched
                split = self.hparams.eval_splits[i].split("_")[-1]
                preds = torch.cat([x["preds"] for x in output]).detach().cpu().numpy()
                labels = torch.cat([x["labels"] for x in output]).detach().cpu().numpy()
                loss = torch.stack([x["loss"] for x in output]).mean()
                self.log(f"val_loss_{split}", loss, prog_bar=True)
                split_metrics = {
                    f"{k}_{split}": v for k, v in self.metric.compute(predictions=preds, references=labels).items()
                }
                self.log_dict(split_metrics, prog_bar=True)
            self.validation_step_outputs.clear()
            return loss

        preds = torch.cat([x["preds"] for x in self.validation_step_outputs]).detach().cpu().numpy()
        labels = torch.cat([x["labels"] for x in self.validation_step_outputs]).detach().cpu().numpy()
        loss = torch.stack([x["loss"] for x in self.validation_step_outputs]).mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log_dict(self.metric.compute(predictions=preds, references=labels), prog_bar=True)
        self.validation_step_outputs.clear()

    def configure_optimizers(self):
        """Prepare optimizer and schedule (linear warmup and decay)"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate)

        if self.hparams.lr_schedule_type == "linear":
            scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.hparams.warmup_steps,
                num_training_steps=self.trainer.estimated_stepping_batches,
            )
        elif self.hparams.lr_schedule_type == "cosine":
            scheduler = get_cosine_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.hparams.warmup_steps,
                num_training_steps=self.trainer.estimated_stepping_batches,
            )
        elif self.hparams.lr_schedule_type == "constant":
            scheduler = get_constant_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.hparams.warmup_steps
            )

        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}

        return [optimizer], [scheduler]


In [5]:
# === Optuna HPO (lr, weight_decay, warmup_steps) ===
import math
import gc
import optuna
import torch
import lightning as L
from lightning.pytorch.loggers import WandbLogger

# --- Fixed choices per your request ---
FIXED = {
    "lr_schedule_type": "cosine",
    "train_batch_size": 16,
    "eval_batch_size": 32,
    "gradient_clip_val": 0.5,
}

# Keep your existing epochs setting (defined in your Week 1 cell)
# If "epochs" is not in scope for any reason, uncomment the next line:
epochs = 3

# Helper: choose the metric to optimize depending on the GLUE task
def _pick_objective_from_metrics(metrics: dict, task_name: str) -> float:
    """
    Returns a single scalar to maximize.
    Priority by task:
      - MRPC/QQP: prefer f1, fall back to accuracy
      - CoLA: matthews_correlation
      - STS-B: pearson
      - SST-2/QNLI/RTE/MNLI: accuracy (or accuracy_m/mm for MNLI)
    If none found, use negative val_loss (to still drive improvement).
    """
    # Normalize keys to plain strings
    m = {str(k): float(v.detach().cpu() if hasattr(v, "detach") else v) for k, v in metrics.items()}

    if task_name.lower() in {"mrpc", "qqp"}:
        for k in ["f1", "eval_f1", "accuracy", "eval_accuracy"]:
            if k in m:
                return m[k]
    if task_name.lower() == "cola":
        for k in ["matthews_correlation", "eval_matthews_correlation"]:
            if k in m:
                return m[k]
    if task_name.lower() == "stsb":
        for k in ["pearson", "eval_pearson"]:
            if k in m:
                return m[k]
    if task_name.lower() == "mnli":
        # Try matched first, then mismatched, else generic accuracy
        for k in ["accuracy_matched", "accuracy_mismatched", "eval_accuracy_matched", "eval_accuracy_mismatched", "accuracy", "eval_accuracy"]:
            if k in m:
                return m[k]
    # Default classification tasks
    for k in ["accuracy", "eval_accuracy", "f1", "eval_f1"]:
        if k in m:
            return m[k]

    # Last resort: minimize val_loss -> maximize -val_loss
    for k in ["val_loss", "val_loss_matched", "val_loss_mismatched"]:
        if k in m:
            return -m[k]

    # If nothing is available, return NaN to signal a bad trial
    return float("nan")


def objective(trial: optuna.Trial) -> float:
    # Clean up between trials
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    L.seed_everything(42, workers=True)

    # --- Sample the three hyperparameters ---
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 5e-4, log=True)
    weight_decay  = trial.suggest_float("weight_decay", 1e-6, 1e-1, log=True)

    # Build the data module first to estimate steps for warmup range
    dm = GLUEDataModule(
        model_name_or_path="distilbert-base-uncased",
        task_name="mrpc",                    # change here if you’re testing another GLUE task
        train_batch_size=FIXED["train_batch_size"],
        eval_batch_size=FIXED["eval_batch_size"],
    )
    dm.setup("fit")

    # Estimate total training steps for a sane warmup upper bound
    train_len = len(dm.dataset["train"])
    steps_per_epoch = math.ceil(train_len / FIXED["train_batch_size"])
    total_steps = steps_per_epoch * epochs
    warmup_upper = max(50, min(1000, total_steps // 2))  # cap to keep search reasonable
    warmup_steps = trial.suggest_int("warmup_steps", 0, warmup_upper, step=max(1, warmup_upper // 20))

    # --- Build the model with fixed + sampled params ---
    model = GLUETransformer(
        model_name_or_path="distilbert-base-uncased",
        num_labels=dm.num_labels,
        eval_splits=dm.eval_splits,
        task_name=dm.task_name,
        learning_rate=learning_rate,
        lr_schedule_type=FIXED["lr_schedule_type"],
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        train_batch_size=FIXED["train_batch_size"],
        eval_batch_size=FIXED["eval_batch_size"],
        gradient_clip_val=FIXED["gradient_clip_val"],
    )

    # Use a disabled W&B logger during search to avoid clutter
    logger = WandbLogger(project="mlops-hpo", name=f"trial-{trial.number}", log_model=False, save_dir=".", mode="disabled")

    trainer = L.Trainer(
        max_epochs=epochs,
        accelerator="auto",
        devices=1,
        logger=logger,
        gradient_clip_val=FIXED["gradient_clip_val"],
        enable_checkpointing=False,
        enable_progress_bar=False,
        log_every_n_steps=25,
    )

    # Train
    trainer.fit(model, datamodule=dm)

    # Pick the metric to maximize
    score = _pick_objective_from_metrics(trainer.callback_metrics, dm.task_name)

    # You can also report intermediate score to Optuna
    if not math.isnan(score):
        trial.report(score, step=0)

    return score


# === Run the study ===
# Set n_trials to the same number of runs you used in week 2 to keep parity.
# If you don't recall, start with 8–12 trials and adjust.
N_TRIALS = 19  # <<— set this to your "week 2" number of runs

study = optuna.create_study(
    direction="maximize",
    study_name="distilbert_glue_hpo_lr_wd_warmup",
)
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

print("Best value:", study.best_value)
print("Best params:", study.best_params)

# === (Optional) Train once with best params and log to W&B normally ===
best = study.best_params
print("\nTraining a final model with the best hyperparameters...")

dm_final = GLUEDataModule(
    model_name_or_path="distilbert-base-uncased",
    task_name="mrpc",   # change here if needed
    train_batch_size=FIXED["train_batch_size"],
    eval_batch_size=FIXED["eval_batch_size"],
)
dm_final.setup("fit")

model_final = GLUETransformer(
    model_name_or_path="distilbert-base-uncased",
    num_labels=dm_final.num_labels,
    eval_splits=dm_final.eval_splits,
    task_name=dm_final.task_name,
    learning_rate=best["learning_rate"],
    lr_schedule_type=FIXED["lr_schedule_type"],
    warmup_steps=best["warmup_steps"],
    weight_decay=best["weight_decay"],
    train_batch_size=FIXED["train_batch_size"],
    eval_batch_size=FIXED["eval_batch_size"],
    gradient_clip_val=FIXED["gradient_clip_val"],
)

final_logger = WandbLogger(project="mlops-hpo", name="best-run", log_model=False)
final_logger.experiment.config.update({
    "model_name": model_final.hparams.model_name_or_path,
    "task": dm_final.task_name,
    "learning_rate": model_final.hparams.learning_rate,
    "lr_schedule_type": model_final.hparams.lr_schedule_type,
    "warmup_steps": model_final.hparams.warmup_steps,
    "weight_decay": model_final.hparams.weight_decay,
    "train_batch_size": model_final.hparams.train_batch_size,
    "eval_batch_size": model_final.hparams.eval_batch_size,
    "gradient_clip_val": model_final.hparams.gradient_clip_val,
    "epochs": epochs,
    "seed": 42,
})

trainer_final = L.Trainer(
    max_epochs=epochs,
    accelerator="auto",
    devices=1,
    logger=final_logger,
    gradient_clip_val=FIXED["gradient_clip_val"],
)
trainer_final.fit(model_final, datamodule=dm_final)

print("Final metrics:", {k: v.item() if hasattr(v, "item") else v for k, v in trainer_final.callback_metrics.items()})

[I 2025-10-14 09:18:26,707] A new study created in memory with name: distilbert_glue_hpo_lr_wd_warmup


  0%|          | 0/19 [00:00<?, ?it/s]

Seed set to 42
/Users/janwahli/Projects/AI/mlops/.venv/lib/python3.13/site-packages/optuna/distributions.py:702: UserWarning: The distribution is specified by [0, 345] and step=17, but the range is not divisible by `step`. It will be replaced by [0, 340].
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


Final metrics: {'val_loss': 0.4079899489879608, 'accuracy': 0.8480392098426819, 'f1': 0.894197940826416}
